In [1]:
###Python Packages###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm
import math as mth

import timeit
import ODE_utils as odeu
from scipy import optimize
import tables
from mpmath import *


In [2]:
###Initial Values###

M0=1.0
Q=0.95
Lambda=0.0000
scalarfield=False
vscalarfield=False


In [3]:
Tol=1
N=50
Nul=100
Nup=10**(40)
scal=10
umax=15
vmax=200
datatype=np.float64


#ru0=5.0
#dr0v=.4
ru0=2.0
#dr0v=0.045
dr0v=1/(4*.375)*(1-2*M0/ru0+Q**2/ru0**2)
#dr0v=0.045

bdytype="stan"
scaltostan=False
solveaffine=False

Elist=[1]
#Elist=[1,2,4,8,16]

#uloc=[1/8,1/4,3/8,1/2,5/8]
#uloc=[1/5,2/5,3/5,4/5,9/10]
uloc=[8.2/10]

In [4]:
###Defining Needed Constants###

scalf=float(scal)
u0=0.0
v0=0.0
rv0=ru0
phiu0=0.0
phiv0=0.0

if M0==0.0:
    dv0=1/N    
else:
    dv0=M0/N
    


def fr(r,M0,Q,Lambda):
    return 1-2*M0/r+Q**2.0/r**2.0-Lambda*r**2.0/3.0

if Lambda>0 or Lambda<0:
    sol = optimize.root(fr,[0.687,1.3,50.0],args=(M0,Q,Lambda), method='hybr')
    rminus=sol.x[0]
    rplus=sol.x[1]
    rcosm=sol.x[2]
else:
    rplus=M0+(M0**2-Q**2)**(.5)
    rminus=M0-(M0**2-Q**2)**(.5)
    rcosm=0.0
    

wHD=True 

print("r+: "+str(rplus))
print("r-: "+str(rminus))
print("rc: "+str(rcosm))

###Predicting Run time###

###timer
start = timeit.default_timer()

Nu=Nul*umax*100#int(umax/du0)
Nv=int(vmax/dv0)
#print("Number of points for lowest iteration is "+str(Nu*Nv)+","+str(Nu)+"X"+str(Nv))
print("Number of points for highest iteration is "+str(Nu*Nv*max(Elist)**2)+","+str(Nu*max(Elist))+"X"+str(Nv*max(Elist)))

numpoints=0.0
for i in range(0,len(Elist)):
    numpoints=Elist[i]**2.0*Nu*Nv+numpoints
print("Total number of points is "+str(int(numpoints)))

predtime=numpoints/(78000)

print("Runtime: about "+str(format(predtime/60,'.2f'))+" minutes")


r+: 1.31224989991992
r-: 0.6877501000800801
rc: 0.0
Number of points for highest iteration is 1500000000,150000X10000
Total number of points is 1500000000
Runtime: about 320.51 minutes


In [5]:
##################################
###Applying Boundary Conditions###

Emax=max([Elist])


rnpf=np.zeros((2,Nv),dtype=datatype)
signpf=np.zeros((2,Nv),dtype=datatype)
phinpf=np.zeros((2,Nv),dtype=datatype)
drnpvf=np.zeros((2,Nv),dtype=datatype)
dsignpvf=np.zeros((2,Nv),dtype=datatype)
dphinpvf=np.zeros((2,Nv),dtype=datatype)
drnpuf=np.zeros((2,Nv),dtype=datatype)
dsignpuf=np.zeros((2,Nv),dtype=datatype)
dphinpuf=np.zeros((2,Nv),dtype=datatype)
massnpf=np.full((2,Nv),M0)#np.zeros((2,Nv))
#drunpv=np.full((1,Nv-1),-1/(4*dr0v)*(1-2*M0/ru0+Q**2/ru0**2))
 


rnpf[0], signpf[0], phinpf[0] , drnpvf[0], dsignpvf[0], dphinpvf[0], drnpuf[0], dsignpuf[0], dphinpuf[0] = odeu.boundaryv(scal,bdytype,Nv,ru0,dr0v,dv0,vmax,M0,Q,Lambda,scalarfield,datatype)
massnpf[0]=M0

print(dsignpuf[0])

Using Standard Coordinates Along V
[0.         0.00080726 0.00161341 ... 0.45288468 0.45288526 0.45288584]


In [6]:
drunptemp=np.empty((Nv))*np.nan
#dsigunp=np.empty((Nu))*np.nan


A=.115
u1=9.5
u2=10

###Applying Propagation Algorithm###

urange=np.array([0.0],dtype=datatype)

massnpf[0][0]=1.0

#drunp[0]=-mth.exp(signpf[0][0])/(4.0*dr0v)*(1-2*M0/ru0+Q**2.0/ru0**2.0-Lambda*ru0**2/3)
#dsigunpvalue=100*np.exp(np.nanmax(signpf))
#drunptemp=drunp[0]
    

i=0
dv=dv0
if M0>0:
    du0=M0/Nul
elif M0==0:
    du0=1/Nul

du=du0

###fix this part (put in front of boundary conditions so it is an initial condition)
#if bdytype=="edd" or bdytype=="fulledd":
    #bdyvalue=1.0
    #dsignpuf[0]=2*(3*Q**2-3*M0*ru0+ru0**4*Lambda)/(ru0*(-3*Q**2+ru0*(6*M0-3*ru0+ru0**3*Lambda)))*drunp[0]
    #dsigunp[0]=2.0*(M0*rnpf[0]-Q**2.0)/(rnpf[0]*(Q**2.0+rnpf[0]*(-2*M0+rnpf[0])))*drunp[0]
#else:
    #bdyvalue=0.0
    #dsigunp[0]=0.0
######
    
atom = tables.Float64Atom()

rnpfile = tables.open_file('rnp.h5', mode='w',encoding="utf8") 
rnp = rnpfile.create_earray(rnpfile.root, 'data', atom, (0, Nv))

signpfile = tables.open_file('signp.h5', mode='w',encoding="utf8") 
signp = signpfile.create_earray(signpfile.root, 'data', atom, (0, Nv))
phinpfile = tables.open_file('phinp.h5', mode='w',encoding="utf8") 
phinp = phinpfile.create_earray(phinpfile.root, 'data', atom, (0, Nv))
massnpfile = tables.open_file('massnp.h5', mode='w',encoding="utf8") 
massnp = massnpfile.create_earray(massnpfile.root, 'data', atom, (0, Nv))
    
 
#rtemp=rnpf[0]   
#print(rtemp)
rnp.append(rnpf[0].reshape(1,Nv))
signp.append(signpf[0].reshape(1,Nv))
phinp.append(phinpf[0].reshape(1,Nv))
massnp.append(massnpf[0].reshape(1,Nv))

#drunp=np.append(drunp,np.nan)
#dsigunp=np.append(dsigunp,np.nan)
#dphinpuf=np.append(dphinpuf,np.nan)
drunptemp=np.nanmax(np.abs(drnpuf[0]))     
dsigunpvalue=np.nanmax(np.abs(dsignpuf[0]))
dphiutemp=np.nanmax(np.abs(dphinpuf[0]))

i=0
TempTol=1 
TempTolv=1
Eres=False
while max(urange)<umax and i<50000:
    
    #rnpf=np.insert(rnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #phinpf=np.insert(phinpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #signpf=np.insert(signpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #massnpf=np.insert(massnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    
    #drunp=np.append(drunp,np.nan)
    #dsigunp=np.append(dsigunp,np.nan)
    #dphinpuf=np.append(dphinpuf,np.nan)
    
    du=du0*Tol*TempTol/max([abs(dsigunpvalue),abs(drunptemp),abs(dphiutemp)])
    #du=abs(du0*Tol*TempTol/mth.exp(np.nanmax(signpf[0,:])))
    #dui=abs(mth.exp(np.nanmax(signpf[0,:]))/(du0*Tol))
    
    if du<1/Nup*1.1:
        du=1/Nup
     
    
    if du>=1/Nul:
        #print('r='+str(rnpf[i][0]))
        if rnpf[1][0]<rplus:
            du=1/(1*Nul)
            #print(rplus)
        else:
            du=1/Nul
    
    
    
    
    print("%.24f" % urange[i])
    print("%.24f" % du)
    
    ###fix this section (dphinpuf is in the wrong form)
    #if urange[i]>u1 and urange[i]<u2 and vscalarfield==True:
        #u=urange[i]+du
        #phinpf[1][0]=A*64*(u-u1)**3.0*(u2-u)**3.0/(u2-u1)**6.0
        #dphinpuf[i]=192*A*(u-u1)**2.0*(u-u2)**2.0*(-2*u+u1+u2)/(u1-u2)**6.0 
    #else:
        #continue
        #phinpf[1][0]=0.0
        #dphinpuf[i+1]=0.0
    
    #print(drunp,dsigunp,dphinpuf)
    
    
    #rnpf[1][0]=rnpf[0][0]+du*drunp[i]
    #drunp[i+1]=drunp[i]+du*(drunp[i]*dsigunp[i]-rnpf[0][0]*dphinpuf[i]**2.0)
    
    #signpf[1][0]=(signpf[0][0]+du*dsigunp[i])*(1.0-bdyvalue)+np.log(abs(1.0-2.0*M0/rnpf[1][0]+Q**2.0/rnpf[1][0]**2.0))*bdyvalue
    #dsigunp[i+1]=2*(3*Q**2-3*M0*rnpf[1][0]+rnpf[1][0]**4*Lambda)/(rnpf[1][0]*(-3*Q**2+rnpf[1][0]*(6*M0-3*rnpf[1][0]+rnpf[1][0]**3*Lambda)))*drunp[i+1]*bdyvalue

    #massnpf[1][0]=M0
    ###Predictor###
    drnpvf[1]=drnpvf[0]+du*odeu.Rfunc(drnpvf[0],drnpuf[0],rnpf[0],signpf[0],Q)
    dsignpvf[1]=dsignpvf[0]+du*odeu.Sigfunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0],signpf[0],Q)
    dphinpvf[1]=dphinpvf[0]+du*odeu.Phifunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0])
    drnpuf[1]=drnpuf[0]+du*odeu.Coneq(drnpuf[0],dsignpuf[0],dphinpuf[0],rnpf[0])#(drnpuf[0]*dsignpuf[0]-rnpf[0]*np.power(dphinpuf[0],2.0))
    
    rnpf[1]=rnpf[0]+du*drnpuf[0]
    signpf[1]=signpf[0]+du*dsignpuf[0]
    phinpf[1]=phinpf[0]+du*dphinpuf[0]
    
    dsignpuf[1][0]=0.0#-1/(4*dsignpvf[1][0])*np.exp()
    dphinpuf[1][0]=0.0
    signpf[1][0]=0.0
    phinpf[1][0]=0.0
        
    for j in range(0,Nv-1):
        dphinpuf[1][j+1]=dphinpuf[1][j]+dv*odeu.Phifunc(drnpvf[1][j],drnpuf[1][j],dphinpuf[1][j],dphinpvf[1][j],rnpf[1][j])         
        #dsignpuf[1][j+1]=dsignpuf[1][j]+dv*odeu.Sigfunc(drnpvf[1][j],drnpuf[1][j],dphinpuf[1][j],dphinpvf[1][j], rnpf[1][j],signpf[1][j],Q)                                                                             
    #Phifuncnp=odeu.Phifunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1])
    Sigfuncnp=odeu.Sigfunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1],signpf[1],Q)
    dsignpuf[1]=dsignpuf[1][0]+dv*(np.cumsum(Sigfuncnp)-Sigfuncnp)
    
    ###Correction###
    drnpvf[1]=drnpvf[0]+1/2*du*(odeu.Rfunc(drnpvf[0],drnpuf[0],rnpf[0],signpf[0],Q)+odeu.Rfunc(drnpvf[1],drnpuf[1],rnpf[1],signpf[1],Q))
    dsignpvf[1]=dsignpvf[0]+1/2*du*(odeu.Sigfunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0],signpf[0],Q)+odeu.Sigfunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1],signpf[1],Q))
    dphinpvf[1]=dphinpvf[0]+1/2*du*(odeu.Phifunc(drnpvf[0],drnpuf[0],dphinpuf[0],dphinpvf[0],rnpf[0])+odeu.Phifunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1]))
    drnpuf[1]=drnpuf[0]+1/2*du*(odeu.Coneq(drnpuf[0],dsignpuf[0],dphinpuf[0],rnpf[0])+odeu.Coneq(drnpuf[1],dsignpuf[1],dphinpuf[1],rnpf[1]))
    
    rnpf[1]=rnpf[0]+1/2*du*(drnpuf[0]+drnpuf[1])
    signpf[1]=signpf[0]+1/2*du*(dsignpuf[0]+dsignpuf[1])
    phinpf[1]=phinpf[0]+1/2*du*(dphinpuf[0]+dphinpuf[1])
    
    dsignpuf[1][0]=0.0#-1/(4*dsignpvf[1][0])*np.exp()
    dphinpuf[1][0]=0.0
    #signpf[1][0]=0.0
    #phinpf[1][0]=0.0
        
    for j in range(0,Nv-1):
        dphinpuf[1][j+1]=dphinpuf[1][j]+1/2*dv*(odeu.Phifunc(drnpvf[1][j],drnpuf[1][j],dphinpuf[1][j],dphinpvf[1][j],rnpf[1][j])+odeu.Phifunc(drnpvf[1][j+1],drnpuf[1][j+1],dphinpuf[1][j+1],dphinpvf[1][j+1],rnpf[1][j+1]))         
        #dsignpuf[1][j+1]=dsignpuf[1][j]+1/2*dv*(odeu.Sigfunc(drnpvf[1][j],drnpuf[1][j],dphinpuf[1][j],dphinpvf[1][j],rnpf[1][j],signpf[1][j],Q)+odeu.Sigfunc(drnpvf[1][j+1],drnpuf[1][j+1],dphinpuf[1][j+1],dphinpvf[1][j+1],rnpf[1][j+1],signpf[1][j+1],Q))    
    Sigfuncnp=odeu.Sigfunc(drnpvf[1],drnpuf[1],dphinpuf[1],dphinpvf[1],rnpf[1],signpf[1],Q)
    dsignpuf[1]=dsignpuf[1][0]+dv*(np.cumsum(Sigfuncnp)-Sigfuncnp)
    
    massnpf[1]=(1+4.0*np.exp(-signpf[1])*drnpuf[1]*drnpvf[1])*rnpf[1]/2.0+np.power(Q,2.0)/(2.0*rnpf[1])
    
    drunptemp=np.nanmax(np.abs(drnpuf[1]))      
    dsigunpvalue=np.nanmax(np.abs(dsignpuf[1]))    
    dphiutemp=np.nanmax(np.abs(dphinpuf[1]))
    #if any(abs((np.exp(signpf[0][:])-np.exp(signpf[1][:]))/np.exp(signpf[0][:]))>0.01):  
    if float(dsigunpvalue*du)>1000:#0.1:
        TempTol=TempTol/2
        Eres=True
    else:
        rnp.append(rnpf[1].reshape(1,Nv))
        signp.append(signpf[1].reshape(1,Nv))
        phinp.append(phinpf[1].reshape(1,Nv))
        massnp.append(massnpf[1].reshape(1,Nv))
    
        #drunp=np.append(drunp,np.nan)
        #dsigunp=np.append(dsigunp,np.nan)
        #dphinpuf=np.append(dphinpuf,np.nan)
    
        rnpf[0]=rnpf[1]
        phinpf[0]=phinpf[1]
        signpf[0]=signpf[1]
        drnpvf[0]=drnpvf[1]
        dsignpvf[0]=dsignpvf[1]
        dphinpvf[0]=dphinpvf[1]
        drnpuf[0]=drnpuf[1]
        dsignpuf[0]=dsignpuf[1]
        dphinpuf[0]=dphinpuf[1]
        massnpf[0]=massnpf[1]
    
        urange=np.append(urange,urange[i]+du)
        
        if Eres==True:
            TempTol=2*TempTol
        else:
            TempTol=1
        Eres=False
        i+=1
    
    #print(TempTol)
    print(drnpuf[1][-1])
    print(dsignpuf[1][-1])
    print(massnpf[1][-1])
    #print(dphinpuf[1][-1])
    print('---')
    
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    #print(urange)
     
    #print(urange)
  
    
    #du=abs(du0*Tol/np.nanmin(drunptemp))

    
    
    
    
    if rnpf[1][0]<0.0 or np.isnan(du):
        break
   
    #dumaxlist=[]
    
    
    #if i>1000:
        #break


0.000000000000000000000000
0.006410711078204735582919
-1.5644367157384873
0.4552649685033031
1.0000000717621031
---
0.006410711078204735582919
0.006392077032837683442379
-1.5690059339456435
0.4572491285867284
1.0000001431059138
---
0.012802788111042419025298
0.006373462192620642403906
-1.573595074563142
0.4592417385569796
1.0000002147234854
---
0.019176250303663062296566
0.006354875000340337413973
-1.5782042224373714
0.4612428299791935
1.0000002866149078
---
0.025531125304003400577901
0.006336315578066346008990
-1.5828334627315248
0.46325243451846887
1.0000003587802726
---
0.031867440882069747454253
0.006317784047061284456726
-1.5874828809266035
0.46527058394030646
1.0000004312196544
---
0.038185224929131032778340
0.006299280527776818759678
-1.5921525628224231
0.46729731011105424
1.0000005039331328
---
0.044484505456907852405379
0.006280805139849732904311
-1.5968425945386255
0.4693326449983163
1.0000005769207707
---
0.050765310596757581840244
0.006262358002098066499241
-1.6015530625156

-1.9497430333475183
0.6217144072858607
1.0000059065442364
---
0.420823413917514932425235
0.005128881000708579093739
-1.9559835247795836
0.6243940975406068
1.0000059974153515
---
0.425952294918223506314803
0.005112517499924690636548
-1.962250922102371
0.6270847954725342
1.0000060885488946
---
0.431064812418148202155521
0.005096188202722780162535
-1.9685453358326372
0.6297865411731995
1.0000061799444602
---
0.436161000620870986654865
0.005079893166783629697425
-1.974866876889501
0.6324993748750249
1.00000627160163
---
0.441240893787654631097439
0.005063632448861780412097
-1.9812156565958565
0.6352233369520207
1.0000063635199843
---
0.446304526236516396764387
0.005047406104786237787674
-1.9875917866797952
0.6379584679205117
1.000006455699097
---
0.451351932341302608531208
0.005031214189461238363299
-1.9939953792760334
0.6407048084398425
1.0000065481385356
---
0.456383146530763839088252
0.005015056756867076331874
-2.0004265469273492
0.6434623993131245
1.0000066408378565
---
0.4613982032876

-2.4833417618652533
0.8489182745670908
1.0000131980007894
---
0.754404854368471933234730
0.004026831970356322884519
-2.4918635007284045
0.8525164873885723
1.0000133065993178
---
0.758431686338828292548442
0.004013060906858208036485
-2.5004213670026316
0.8561290537066094
1.0000134154160512
---
0.762444747245686449410584
0.003999325926408736804618
-2.509015504785013
0.8597560260290414
1.0000135244502204
---
0.766444073172095174939500
0.003985627024196830570091
-2.5176460586996527
0.8633974570674924
1.0000136337010546
---
0.770429700196291977754015
0.003971964194667193526200
-2.526313173899724
0.8670533997382721
1.0000137431677714
---
0.774401664390959143524640
0.003958337431524207132572
-2.5350169960695266
0.8707239071634508
1.0000138528495977
---
0.778360001822483327238444
0.003944746727735838447149
-2.5437576714265533
0.8744090326718759
1.0000139627457447
---
0.782304748550219142266826
0.003931192075537582285294
-2.552535346723565
0.8781088298001547
1.0000140728554339
---
0.78623594062

-3.1984150334941224
1.1480330028860426
1.0000215262505994
---
1.011275186242805768088715
0.003126548585871126613445
-3.2099399422934614
1.1528121189107898
1.0000216484485114
---
1.014401734828676904243139
0.003115323083850324824284
-3.221512833541686
1.157609953818827
1.0000217708037955
---
1.017517057912527222995891
0.003104131666303542279317
-3.233133895154621
1.1624265775936296
1.0000218933155738
---
1.020621189578830856348191
0.003092974285719076506934
-3.2448033157506146
1.167262060504862
1.0000220159829583
---
1.023714163864549941962423
0.003081850894154032193678
-3.2565212846534113
1.1721164731097458
1.0000221388050765
---
1.026796014758704078673190
0.003070761443238744729106
-3.26828799189504
1.1769898862542547
1.00002226178105
---
1.029866776201942890622831
0.003059705884181196378080
-3.2801036282187144
1.1818823710744981
1.0000223849099938
---
1.032926482086124142512062
0.003048684167771424213117
-3.291968385081747
1.1867939989979681
1.000022508191032
---
1.035975166253895629

-4.1628239567101195
1.5444617968195757
1.0000306748386947
---
1.209672863219869221040881
0.002402215444129182295163
-4.1783289580409155
1.5507860062157275
1.0000308061593206
---
1.212075078663998439765237
0.002393301269579472916049
-4.193897452751662
1.5571347861791103
1.0000309375762353
---
1.214468379933577812934686
0.002384416908772743568662
-4.209529687503003
1.5635082308546098
1.0000310690886458
---
1.216852796842350592498860
0.002375562293737325594845
-4.22522590990074
1.569906434777513
1.00003120069577
---
1.219228359136087957992345
0.002366737356354733749775
-4.240986368499751
1.5763294928750542
1.0000313323968186
---
1.221595096492442689140034
0.002357942028362967814420
-4.256811312807929
1.5827775004680869
1.0000314641910113
---
1.223953038520805725042351
0.002349176241359798595482
-4.2727009932901305
1.5892505532726737
1.00003159607756
---
1.226302214762165609940325
0.002340439926806028771850
-4.2886556613721485
1.5957487474018053
1.000031728055693
---
1.22864265468897171373

...

-2.0261504742546433e-38
-422.0604718285761
1.0002700579081465
---
5.098657013055063913498088
0.000023693287259702432840
-2.0059861553315986e-38
-422.23392625343257
1.0002700578899102
---
5.098680706342323176727405
0.000023683554016447782886
-1.9860225136734858e-38
-422.40739785850667
1.000270057871674
---
5.098704389896339783661006
0.000023673827803910026346
-1.966257552107179e-38
-422.58088664021585
1.0002700578534376
---
5.098728063724143666490818
0.000023664108614817619148
-1.9466892933358904e-38
-422.75439259497915
1.0002700578352013
---
5.098751727832758540159830
0.000023654396441908822474
-1.9273157797413544e-38
-422.92791571921816
1.000270057816965
---
5.098775382229200126005253
0.000023644691277931628221
-1.90813507318798e-38
-423.1014560093557
1.0002700577987287
---
5.098799026920477928115361
0.000023634993115643826764
-1.8891452548289552e-38
-423.27501346181674
1.0002700577804924
---
5.098822661913593456972649
0.000023625301947812922250
-1.8703444249142767e-38
-423.4485880730

-9.762872707669109e-39
-434.76757364499485
1.0002700565768985
---
5.100361473573975068518394
0.000023000795381684560327
-9.665712706149077e-39
-434.9422729253191
1.0002700565586622
---
5.100384474369357157286231
0.000022991556862804709069
-9.569519645752792e-39
-435.11698912914943
1.000270056540426
---
5.100407465926219963137100
0.000022982324868569648462
-9.474283903372671e-39
-435.2917222530356
1.0002700565221896
---
5.100430448251088932920538
0.000022973099392381710130
-9.379995951672042e-39
-435.46647229352897
1.0002700565039535
---
5.100453421350481519880304
0.000022963880427651926422
-9.286646358132008e-39
-435.64123924718325
1.0002700564857172
---
5.100476385230908960011220
0.000022954667967799969420
-9.194225784107792e-39
-435.8160231105534
1.000270056467481
---
5.100499339898877160237589
0.000022945462006254188213
-9.102724983894489e-39
-435.9908238801961
1.0002700564492446
---
5.100522285360883145699518
0.000022936262536451578403
-9.01213480380209e-39
-436.1656415526704
1.000

-4.751477775107474e-39
-447.3889743315505
1.0002700552638877
---
5.101994299648743513841964
0.000022351914270889498647
-4.704191323698923e-39
-447.5648833891326
1.0002700552456514
---
5.102016651563014093540005
0.000022343129166605236942
-4.6573754672435774e-39
-447.7408091262207
1.000270055227415
---
5.102038994692180473577992
0.000022334350133317738406
-4.611025522349793e-39
-447.91675153948773
1.000270055209179
---
5.102061329042313353454574
0.000022325577165020170638
-4.565136852235651e-39
-448.0927106256081
1.0002700551909427
---
5.102083654619478103597885
0.000022316810255713426177
-4.519704866265083e-39
-448.26868638125785
1.0002700551727064
---
5.102105971429733877187118
0.000022308049399406142832
-4.474725019488624e-39
-448.4446788031155
1.00027005515447
---
5.102128279479133610152530
0.000022299294590114615588
-4.4301928121887274e-39
-448.62068788786013
1.0002700551362338
---
5.102150578773724021175440
0.000022290545821862898254
-4.3861037894296075e-39
-448.7967136321728
1.00

-2.382935148250347e-39
-459.565656525725
1.0002700540055858
---
5.103516284855167839396017
0.000021759676464075015326
-2.359220399286589e-39
-459.74270874966953
1.0002700539873497
---
5.103538044531632067446481
0.000021751296561496994179
-2.3357416586135134e-39
-459.91977742737544
1.0002700539691134
---
5.103559795828193301758802
0.000021742922332969405415
-2.3124965774716333e-39
-460.09686255562696
1.000270053950877
---
5.103581538750526114256445
0.000021734553772991600134
-2.2894828304765114e-39
-460.2739641312105
1.0002700539326408
---
5.103603273304298859613937
0.000021726190876069834449
-2.2666981153861295e-39
-460.45108215091324
1.0002700539144045
---
5.103624999495174563435285
0.000021717833636717333859
-2.2441401528705696e-39
-460.6282166115243
1.0002700538961684
---
5.103646717328810922253979
0.000021709482049454228877
-2.2218066862839907e-39
-460.80536750983435
1.0002700538779319
---
5.103668426810860303532991
0.000021701136108807551640
-2.199695481438873e-39
-460.98253484263

KeyboardInterrupt: 

In [7]:
rnpfile.close()
signpfile.close()
phinpfile.close()
massnpfile.close()

In [8]:
Nu=i+2

print(Nu)
#rnpf[i+1][j+1]=answer[0]
#phinpf[i+1][j+1]=answer[1]
#signpf[i+1][j+1]=answer[2]
#massnpf[i+1][j+1]=answer[3]
#drunp[i+1][j+1]=answer[4]
#dsigunp[i+1][j+1]=answer[5]

13313


In [9]:
#print(rnpf[1,0])

In [10]:
#print(dnu.x4giveralt(0,0,du,dv,rnpf,phinpf,signpf,Q,Lambda))

In [11]:
#print(rnpf)

In [12]:
vrange=np.arange(0,vmax,dv0)
#print(urange)

In [13]:
switch=0.0
np.save('rnputil',np.array([Nu*Nv,Nu,Nv,ru0,dr0v,M0,Q,Lambda,wHD,switch]))
np.save('urange',urange)
np.save('vrange',vrange)

In [14]:
stop = timeit.default_timer()

In [15]:
acttime=stop - start

numpoints=Nu*Nv



print(str(acttime)+' seconds')
print(str((acttime)/60)+' minutes')
print(str((acttime)/3600)+' hours')

print(str(numpoints)+' points')

print(str((numpoints)/(stop - start))+' points per second') 

exttimef=(acttime-predtime)/60

exttime=format(abs((acttime-predtime)/60),'.2f')
if exttimef>0.0:
    print("Took "+str(exttime)+" more minutes")
if exttimef<0.0:
    print("Took "+str(exttime)+" less minutes")
if exttimef==0.0:
    print("Took exactly the right time!!!!")

678.1834031870021 seconds
11.303056719783369 minutes
0.18838427866305615 hours
133130000 points
196303.83075489503 points per second
Took 309.21 less minutes


conv=False
j=0
tempanswer=[]
    while conv==False:
        dv=TempTolv*dv0
        jcount=1
        while j<Nv-1:
            j1=mth.ceil(jcount*TempTolv)
            j2=mth.floor(jcount*TempTolv)
            answer=dnu.x4giveralt(0,j+mth.ceil((jcount-1)*TempTolv),du,dv*TempTolv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
            rnpf[1][j+j1]=answer[0]
            phinpf[1][j+j1]=answer[1]
            signpf[1][j+j1]=answer[2]
            massnpf[1][j+j1]=answer[3]
            #drunptemp[j+1]=answer[4]
            dsigunpv=answer[5]
            jcount=(jcount+1)**(0**j2)
            j=j+j2
        
        if np.abs(tempanswer[0]-answer[0])/answer[0]>0.01 or not tempanswer:
            tempanswer=answer
            TempTol=TempTolv/2
            
        else:
            conv=True
            